In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!ls /content/drive/
%cd "/content/drive/My Drive"
%cd "Colab Notebooks/nthudl/final_project"
!ls

In [ ]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.models import load_model
import argparse
import time
import os
from datetime import datetime as dt

In [ ]:
# 匯出成csv
def output(filepath, data):
    df_SAMPLE = pd.DataFrame.from_dict(data)
    df_SAMPLE.to_csv(filepath, index=False)
    print('Success output to ' + filepath)


def RMSE(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
# argparse
parser = argparse.ArgumentParser(description='Final Project')
parser.add_argument('--epochs', default='100', type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('-b', '--batch-size', default='32', type=int,
                    metavar='N',
                    help='mini-batch size (default: 32), this is the total '
                         'batch size of all GPUs on the current node when '
                         'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--opt', type=str, default='SGD',
                        help='SGD or Adam (default: SGD)')
parser.add_argument('--lr', '--learning-rate', default='0.1', type=float,
                    metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--save', default='', type=str, metavar='PATH',
                    help='path to latest dir (default: none)')
parser.add_argument('--p', '--predict', dest='predict', action='store_true',
                    help='evaluate model on test set')

In [ ]:
# today = dt.now()

# SGD Setting
# args = parser.parse_args(args=['--epochs', '50', '--batch-size', '64', '--opt', 'SGD', '--lr', '0.001', '--resume', './workspace/my_model.h5', '--save', '.'])
# predict
# args = parser.parse_args(args=['--epochs', '50', '--batch-size', '64', '--opt', 'SGD', '--lr', '0.001', '--resume', './workspace/my_model.h5', '--save', '.','--p'])
# Adam Setting
args = parser.parse_args(args=['--epochs', '6', '--batch-size', '128', '--opt', 'Adam', '--lr', '0.0001', '--resume', './workspace/my_model.h5', '--save', '.'])
# predict
# args = parser.parse_args(args=['--epochs', '30', '--batch-size', '64', '--opt', 'Adam', '--lr', '0.0001', '--resume', './workspace/my_model.h5', '--save', '.','--p'])
DIR =  './NN/0117n6v8_epochs_' + str(args.epochs) + '_batch_' + str(args.batch_size) + '_' + str(args.opt) + '_lr_' + str(args.lr)

if not os.path.exists(DIR+'/logs/'):
    os.makedirs(DIR+'/logs/')
if not os.path.exists(DIR+'/result/'):
    os.makedirs(DIR+'/result/')
if not os.path.exists(DIR+'/workspace/'):
    os.makedirs(DIR+'/workspace/')
    
with open(DIR+'/logs/args.txt', 'w') as f:
    f.write(str(args))

In [ ]:
x_train = pd.read_csv('v8/x_train.csv', header=0, low_memory=False)
y_train = pd.read_csv('v8/y_train.csv', header=0, low_memory=False)
x_test = pd.read_csv('v8/x_test.csv', header=0, low_memory=False)
y_test = pd.read_csv('v8/y_test.csv', header=0, low_memory=False)
print('x_train.shape= {}, y_train.shape= {}'.format(x_train.shape, y_train.shape))
print('x_test.shape= {}, y_test.shape= {}'.format(x_test.shape, y_test.shape))

x_train.set_index('ID', inplace=True)
y_train.set_index('ID', inplace=True)
x_test.set_index('ID', inplace=True)
y_test.set_index('ID', inplace=True)

print('x_train.shape= {}, y_train.shape= {}'.format(x_train.shape, y_train.shape))
print('x_test.shape= {}, y_test.shape= {}'.format(x_test.shape, y_test.shape))

In [ ]:
# predict
if args.predict:
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading model '{}'".format(args.resume))
            model = load_model(args.resume)
        else:
            print("=> no model found at '{}'".format(args.resume))
    model = load_model(args.resume)
    result = model.predict(x_test)
    y_test['乳量'] = result
    y_test.reset_index(inplace=True)
    print(y_test.head(n=3))
    output(args.save +'/result/submission(NN).csv', y_test)

In [ ]:
# train
if args.opt == 'Adam':
    optim = Adam(learning_rate=args.lr, beta_1=0.9, beta_2=0.999, epsilon=1e-5)
elif args.opt == 'SGD':
    optim = SGD(learning_rate=args.lr)
else:
    optim = None

model = Sequential()
# 7-layer nn
# model.add(Dense(4096, input_dim = x_train.shape[1], activation='relu'))
# model.add(Dense(2048, activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(1))
# 6-layer nn
model.add(Dense(2048, input_dim = x_train.shape[1], activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
# 5-layer nn
# model.add(Dense(1024, input_dim = x_train.shape[1], activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(1))
model.compile(loss=RMSE, optimizer=optim, metrics=[RMSE])
model.summary()
history = model.fit(x_train, y_train, batch_size=args.batch_size, epochs=args.epochs, validation_split=0.005, verbose=1)

In [ ]:
# save model
model.save(DIR + '/workspace/my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
print(history.history.keys())
fig=plt.figure(figsize=(15,8))
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()
fig.savefig(DIR+'/result/loss.png',format='png')

In [ ]:
result = model.predict(x_test)
y_test['乳量'] = result
y_test=y_test.sort_values(by=['ID'])
y_test.reset_index(inplace=True)
print(y_test.head(n=3))
output(DIR +'/result/n6v8_' + str(args.epochs) + '_' + str(args.batch_size) + '_' + str(args.lr)+'.csv', y_test)